In [ ]:
!pip install -U keras-tuner


Requirement already up-to-date: keras-tuner in /usr/local/lib/python3.6/dist-packages (1.0.1)


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import kerastuner
from kerastuner.tuners import RandomSearch,BayesianOptimization


In [ ]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape(60000,28,28,1).astype('float32')/255.0
x_test = x_test.reshape(10000,28,28,1).astype('float32')/255.0

y_train = y_train.astype('int64')
y_test = y_test.astype('int64')


## Hyper parameters 
##### Learning Rate
##### No. of dense layers
##### No.of dense nodes
##### Dropout rate
##### Conv Filters

In [ ]:
def model(hp):
  CNN_Filter1 = hp.Int('layer1_filters', 16, 64, 8)
  CNN_Filter2 = hp.Int('layer2_filters', 32, 128, 16)
  CNN_Filter3 = hp.Int('layer3_filters', 32, 256, 32)
  N_Layers = hp.Int('layers', 1, 3)
  N_Dims = hp.Int('dims',  8, 32, 8) 
  Learning_Rate = hp.Choice('learning_rate', [0.001, 0.0005, 0.0001])
  Dropout_Rate = hp.Float('dropout_rate', 0.1, 0.5, 5)

  model_input=tf.keras.layers.Input(shape=(28,28,1))
  layer=tf.keras.layers.Conv2D(CNN_Filter1,3,activation='relu',\
                              kernel_initializer='uniform')(model_input)
  layer=tf.keras.layers.MaxPooling2D(2,1,'same')(layer)

  layer=tf.keras.layers.Conv2D(CNN_Filter2,3,activation='relu')(layer)
  layer=tf.keras.layers.MaxPooling2D(2,1,'same')(layer)
  layer=tf.keras.layers.Dropout(Dropout_Rate)(layer)

  layer=tf.keras.layers.Conv2D(CNN_Filter3,3,activation='relu')(layer)                             
  layer=tf.keras.layers.MaxPooling2D(2,1,'same')(layer)

  layer=tf.keras.layers.Flatten()(layer)
  for i in range(N_Layers):
    layer=tf.keras.layers.Dense(N_Dims,activation='relu')(layer)
  model_output=tf.keras.layers.Dense(10,activation='softmax')(layer)
  model=tf.keras.models.Model(model_input,model_output)

  model.compile(optimizer=tf.keras.optimizers.Adam(Learning_Rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']) 
  return model

# Random Search

In [ ]:
tuner = RandomSearch(
    model,
    objective='val_accuracy',
    max_trials=1,
    executions_per_trial=1,
    directory='/content/output',
    project_name='cnn_best_model')



INFO:tensorflow:Reloading Oracle from existing project /content/output/cnn_best_model/oracle.json


In [ ]:
tuner.search(x_train, y_train, epochs=5,validation_data=(x_test,y_test))

Epoch 1/5
1875/1875 [==============================] - 737s 393ms/step - loss: 0.1702 - accuracy: 0.9465 - val_loss: 0.0568 - val_accuracy: 0.9815
Epoch 2/5
1875/1875 [==============================] - 747s 398ms/step - loss: 0.0550 - accuracy: 0.9837 - val_loss: 0.0383 - val_accuracy: 0.9875
Epoch 3/5
1875/1875 [==============================] - 747s 398ms/step - loss: 0.0394 - accuracy: 0.9876 - val_loss: 0.0332 - val_accuracy: 0.9881
Epoch 4/5
1875/1875 [==============================] - 748s 399ms/step - loss: 0.0303 - accuracy: 0.9908 - val_loss: 0.0290 - val_accuracy: 0.9907
Epoch 5/5
1875/1875 [==============================] - 747s 399ms/step - loss: 0.0242 - accuracy: 0.9924 - val_loss: 0.0297 - val_accuracy: 0.9909


INFO:tensorflow:Oracle triggered exit


### Best params and model

In [ ]:
tuner.get_best_hyperparameters()[0].values

{'dims': 24,
 'dropout_rate': 0.1,
 'layer1_filters': 64,
 'layer2_filters': 112,
 'layer3_filters': 64,
 'layers': 2,
 'learning_rate': 0.0005}

In [ ]:
models = tuner.get_best_models()[0].summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 26, 26, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 112)       64624     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 112)       0         
_________________________________________________________________
dropout (Dropout)            (None, 24, 24, 112)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 22, 64)        64576 

In [ ]:
tuner.results_summary()

In [ ]:
!git clone https://github.com/keras-team/keras-tuner.git
!cd keras-tuner
!pip install .

Cloning into 'keras-tuner'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 6412 (delta 0), reused 1 (delta 0), pack-reused 6402
Receiving objects: 100% (6412/6412), 1.99 MiB | 11.59 MiB/s, done.
Resolving deltas: 100% (4478/4478), done.
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


# Bayesian Optimization

In [ ]:
bayesian_tuner = BayesianOptimization(
    model,
    objective='val_accuracy',
    max_trials=1,
    executions_per_trial=1,
    directory='/content/output_bayesian',
    project_name='cnn_best_model')

INFO:tensorflow:Reloading Oracle from existing project /content/output_bayesian/cnn_best_model/oracle.json
INFO:tensorflow:Reloading Tuner from /content/output_bayesian/cnn_best_model/tuner0.json


In [ ]:
bayesian_tuner.search(x_train, y_train, epochs=5,validation_data=(x_test,y_test))

Epoch 1/5
1875/1875 [==============================] - 926s 494ms/step - loss: 0.1476 - accuracy: 0.9549 - val_loss: 0.0753 - val_accuracy: 0.9760
Epoch 2/5
1875/1875 [==============================] - 921s 491ms/step - loss: 0.0482 - accuracy: 0.9853 - val_loss: 0.0505 - val_accuracy: 0.9842
Epoch 3/5
1875/1875 [==============================] - 922s 492ms/step - loss: 0.0361 - accuracy: 0.9892 - val_loss: 0.0404 - val_accuracy: 0.9881
Epoch 4/5
1875/1875 [==============================] - 915s 488ms/step - loss: 0.0280 - accuracy: 0.9915 - val_loss: 0.0308 - val_accuracy: 0.9906
Epoch 5/5
1875/1875 [==============================] - 902s 481ms/step - loss: 0.0222 - accuracy: 0.9930 - val_loss: 0.0462 - val_accuracy: 0.9874


INFO:tensorflow:Oracle triggered exit


In [ ]:
bayesian_tuner.get_best_hyperparameters()[0].values

{'dims': 32,
 'dropout_rate': 0.1,
 'layer1_filters': 40,
 'layer2_filters': 80,
 'layer3_filters': 224,
 'layers': 2,
 'learning_rate': 0.001}

In [ ]:
models = bayesian_tuner.get_best_models()[0].summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 40)        400       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 26, 26, 40)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 80)        28880     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 80)        0         
_________________________________________________________________
dropout (Dropout)            (None, 24, 24, 80)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 22, 224)       161504

In [ ]:
bayesian_tuner.results_summary()